In [1]:
from multiprocessing import Pool
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
import glob
import argparse
import os
import subprocess
import datetime
import shutil

The set_paths(sub) method assigns variables for our directory paths.
[input (BIDS), output(anat, func, motion_assesment)]
It takes the subject ID as an argument, and **it is called from get_subjects()


In [ ]:
def set_paths(sub):
    global func_output_path
    global anat_output_path
    global func_input_path
    global motion_assessment_path

    if arglist["SES"] == False:
        out_dir = os.path.join(derivatives_dir, sub)
        func_input_path=os.path.join(input_dir,sub,"fmriprep", sub, "func")
    else:
        out_dir = os.path.join(derivatives_dir, sub, arglist["SES"])
        func_input_path=os.path.join(input_dir,sub,arglist["SES"], "fmriprep", sub,arglist["SES"], "func")
    
    
    anat_output_path=os.path.join(out_dir, 'anat')
    func_output_path=os.path.join(out_dir,'func')
    motion_assessment_path=os.path.join(out_dir,'func','motion_assessment')

The get_subjects() method gets an input directory, the BIDS path, we ask for the "top level" BIDS directory, where the subjects are listed, ~/STUDYNAME. For the output directory we ask for you to provide the location where you would like your derivatives directory to be, or the path to its location, however do not include the derivatives directory.

In [ ]:
def get_subjects():
    # Get subjects from input directory
    sub_dir=glob.glob(os.path.join(input_dir, 'sub*'))
    for path in sub_dir:
        sub = path.split("/")[-1]
        subjects.append(sub)
    
    for sub in sorted(subjects):
        set_paths(sub)
        if args.STRIP == True:
            skull_strip(sub)
        if args.MOCO == True:
            fd_check(sub)


